Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [23]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy
from tqdm import tqdm
from queue import PriorityQueue

## The *Nim* and *Nimply* classes

In [2]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [16]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [4]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [5]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [6]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.5}


In [10]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)

# Use a cache to improve computation time of this function
#analize_cache = dict()
def analize(raw: Nim) -> dict:
    # global analize_cache
    # cached_value = analize_cache.get(raw)
    # if cached_value is not None:
    #     key = list(analize_cache.keys())[list(analize_cache.values()).index(cached_value)]
    #     print(raw, key)
    #     return cached_value
    # else :
        cooked = dict()
        cooked["possible_moves"] = dict()
        for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
            tmp = deepcopy(raw)
            tmp.nimming(ply)
            cooked["possible_moves"][ply] = nim_sum(tmp)
        #analize_cache[raw] = cooked
        #print(raw, "added in cache -->", cooked)
        return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


## Oversimplified match

In [11]:
logging.getLogger().setLevel(logging.INFO)

strategy = (pure_random, optimal)

nim = Nim(5)
logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")


INFO:root:init : <1 3 5 7 9>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=3)
INFO:root:status: <1 0 5 7 9>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=1)
INFO:root:status: <1 0 4 7 9>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=5)
INFO:root:status: <1 0 4 7 4>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=3)
INFO:root:status: <1 0 4 4 4>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=2)
INFO:root:status: <1 0 2 4 4>
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=2)
INFO:root:status: <1 0 2 4 2>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=2)
INFO:root:status: <1 0 2 4 0>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=4)
INFO:root:status: <1 0 2 0 0>
INFO:root:ply: player 0 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 0 2 0 0>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=1)
INFO:root:status: <0 0 1 0 0>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=1)
INFO:root:status: <0 0

## Automatic comparison of strategies

In [40]:
# Run 100 games with the given parameters and print number of victories for each player
def compare_strategies(plyr1_strat, plyr2_strat, nim_size) :
    strategy = (plyr1_strat, plyr2_strat)
    plyr1_victories = 0
    plyr2_victories = 0
    for _ in tqdm(range(1000)) :
        nim = Nim(nim_size)
        player = 0
        while nim:
            ply = strategy[player](nim)
            nim.nimming(ply)
            player = 1 - player
        if player == 0 :
            plyr1_victories += 1
        else :
            plyr2_victories += 1
    
    print("Victories -> Player 1 :", plyr1_victories, ", Player 2 :", plyr2_victories)

compare_strategies(pure_random, optimal_rule_based1, 3)

100%|██████████| 1000/1000 [00:03<00:00, 272.91it/s]

Victories -> Player 1 : 87 , Player 2 : 913


## First task : Improve rule-based agent

In [13]:
def optimal_rule_based1(state : Nim) -> Nimply :
    # We keep the basic idea : we first have to filter the good moves before selecting one
    analysis = analize(state)
    good_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
   
    # If there is no good move, we can only select one random bad move
    if not good_moves:
        good_moves = list(analysis["possible_moves"].keys())
        ply = random.choice(good_moves)
        return ply
    
    # The best move one can make is the one that lets no good move to their opponent
    best_moves = list()
    for ply in good_moves :
        tmp_nim = deepcopy(state)
        tmp_nim.nimming(ply)
        opponent_moves = analize(tmp_nim)["possible_moves"]
        good_opponent_moves = [ply for ply,ns in opponent_moves.items() if ns != 0]
        if not good_opponent_moves :
            best_moves.append(ply)
    
    # If there are best moves, the agent can randomly choose one of them
    if len(best_moves) > 0 :
        ply = random.choice(best_moves)
        return ply
    
    # If there are no best moves, agent can only choose a good move
    ply = random.choice(good_moves)
    return ply

In [24]:
def optimal_rule_based2(state : Nim) -> Nimply :
    # We keep the basic idea : we first have to filter the good moves before selecting one
    analysis = analize(state)
    good_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
   
    # If there is no good move, we can only select one random bad move
    if not good_moves:
        good_moves = list(analysis["possible_moves"].keys())
        ply = random.choice(good_moves)
        return ply
    
    # The possible moves are sorted regarding the number of good moves they let to the opponent
    sorted_good_moves = PriorityQueue()
    for ply in good_moves :
        tmp_nim = deepcopy(state)
        tmp_nim.nimming(ply)
        opponent_moves = analize(tmp_nim)["possible_moves"]
        good_opponent_moves = [ply for ply,ns in opponent_moves.items() if ns != 0]
        sorted_good_moves.put((len(good_opponent_moves), ply))
    
    # We always choose the move that lets the lesser good moves to the opponent
    _, ply = sorted_good_moves.get()
    return ply

In [14]:
logging.getLogger().setLevel(logging.INFO)

strategy = (optimal, optimal_rule_based1)

nim = Nim(5)
logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")


INFO:root:init : <1 3 5 7 9>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=7)
INFO:root:status: <1 3 5 0 9>
INFO:root:ply: player 1 plays Nimply(row=1, num_objects=3)
INFO:root:status: <1 0 5 0 9>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=2)
INFO:root:status: <1 0 3 0 9>
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=2)
INFO:root:status: <1 0 3 0 7>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=3)
INFO:root:status: <1 0 3 0 4>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=2)
INFO:root:status: <1 0 1 0 4>
INFO:root:ply: player 0 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 0 1 0 4>
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=4)
INFO:root:status: <0 0 1 0 0>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=1)
INFO:root:status: <0 0 0 0 0>
INFO:root:status: Player 1 won!
